<h1>Training and Benchmarking a Model Using Model Zoo</h1>

This notebook provides a step-by-step guide on how to use the **STM32AI model zoo** to train, quantize, and benchmark an image classification model. The resulting model can be deployed on STM32 targets, making it ideal for edge computing applications. This notebook can be used in both Google Colab and Jupyter Notebook environments, providing flexibility for users who prefer different development environments.

## License of the Jupyter Notebook

This software component is licensed by ST under BSD-3-Clause license,
the "License";

You may not use this file except in compliance with the
License.

You may obtain a copy of the License at: https://opensource.org/licenses/BSD-3-Clause

Copyright (c) 2023 STMicroelectronics. All rights reserved

<div style="border-bottom: 3px solid #273B5F">
<h2>Table of content</h2>
<ul style="list-style-type: none">
<li><a href="#install">1. Install necessary packages</a>
<li><a href="#config">2. Configure environment variables to access STM32Cube.AI Developer Cloud Services</a></li>
<li><a href="#upload">3. Upload the dataset</h2> </a></li>
<li><a href="#training">4. Training and Benchmarking the Model</a></li>
<li><a href="#results">5. Results</a></li>
  </ul>
</ul>
</div>

<div id="install">
    <h2>1. Install necessary packages</h2>
</div>

To get started, upload the model zoo package and clone the repository using the following command:

In [ ]:
!git clone https://github.com/STMicroelectronics/stm32ai-modelzoo.git

Or, you can upload a lighter version of STM32 model zoo by following these steps:
- On your local PC clone STM32AI model zoo git using the following command:
```
git clone https://github.com/STMicroelectronics/stm32ai-modelzoo.git
```
- Delete the .git directory.

- For image classification use-case, you can keep only the folders 'image_classification' and 'common', as well as the file 'requirements.txt', then delete the rest.

- Zip the repository as stm32ai-modelzoo.zip, and upload **stm32-modelzoo.zip** in your workspace.

- Then uncomment and run the cell below to unzip the folder.

In [ ]:
# import zipfile
# with zipfile.ZipFile('stm32ai-modelzoo.zip', 'r') as zip_ref:
#     zip_ref.extractall('')

Next, run the following command to install the required packages

In [ ]:
!pip install -r stm32ai-modelzoo/requirements.txt

<div id="config">
    <h2>2. Configure environment variables to access STM32Cube.AI Developer Cloud Services</h2>
</div>
Set environment variables with your credentials to acces STM32Cube.AI Developer Cloud Services.

If you don't have an account yet go to: https://stm32ai-cs.st.com/home and click on sign in to create an account.

Then set the environment variables below with your credentials.


In [ ]:
import getpass
import os

os.environ['stmai_username'] = 'xxx.yyy@st.com'
print('Enter you password')
password = getpass.getpass()
os.environ['stmai_password'] = password

<div id="upload">
    <h2>3. Upload the dataset</h2>
</div>
The dataset can be uploaded as a zip archive named **dataset.zip** under the directory '/content/stm32ai-modelzoo/image_classification/datasets' (in Colab workspace='/content/').

The zip file shall contain a directory named "dataset" with one sub-directory per category, with images inside as below:

```bash
dataset_root_directory/
   class_a/
      a_image_1.jpg
      a_image_2.jpg
   class_b/
      b_image_1.jpg
      b_image_2.jpg
```
Other dataset formats are not supported. The only exceptions are the Cifar10/Cifar100 datasets. For these datasets, the official format in batches is supported.

The split between training and validation sets is done automatically by the scripts. However, it is also possible to upload specific training, validation, and test sets by defining specific paths in the user_config.yaml file.

In this tutorial we are going to use the flower dataset that can be downloaded directly from tensorflow repository: https://www.tensorflow.org/datasets/catalog/tf_flowers (Creative Commons By-Attribution License 2.0)

In [ ]:
dataset_name = 'tf_flowers' #@param ["custom", "tf_flowers"]
%cd stm32ai-modelzoo/image_classification

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import zipfile

# Define the path to the dataset directory
if dataset_name == 'tf_flowers':
    path = 'datasets/flower_photos'
    !wget http://download.tensorflow.org/example_images/flower_photos.tgz -P datasets
    !tar -xf datasets/flower_photos.tgz -C datasets
else:
    path = 'datasets/dataset'
    with zipfile.ZipFile('datasets/dataset.zip', 'r') as zip_ref:
        zip_ref.extractall('datasets')

# Get the list of class names
class_names = sorted([name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))])
num_classes = len(class_names)
print(f"Classes: {class_names}")
print(f"Introducing samples from each class...")

# Print a photo from each class
fig, axs = plt.subplots(1, num_classes, figsize=(4*num_classes, 4))
for i, class_name in enumerate(class_names):
    class_path = os.path.join(path, class_name)
    image_files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f)) and f.endswith('.jpg')]
    if len(image_files) == 0:
        img = np.zeros((224, 224, 3))
    else:
        img_path = os.path.join(class_path, random.choice(image_files))
        img = mpimg.imread(img_path)
    axs[i].imshow(img)
    axs[i].set_title(class_name)
    axs[i].axis('off')
plt.show()

<div id="training">
    <h2>4. Training and Benchmarking the Model</h2>
</div>

The STM32 model zoo is an invaluable resource that provides a wide range of use cases, including image classification, object detection, audio event detection, hand posture, and human activity recognition. The model zoo offers various services, including training, evaluation, prediction, deployment, quantization, benchmarking, and chained services. These services, such as chain_tbqeb, chain_tqe, chain_eqe, chain_qb, chain_eqeb, and chain_qd, are thoroughly explained in their respective readmes.

In this section, we will demonstrate how to train, quantize, evaluate, and benchmark a classification model using the chain_tbqeb service. We will use the MobileNet v2 0.35 model from the model zoo as an example, but you can also use your own custom model. To accomplish this, we will use the `user_config.yaml` file as a configuration file to specify the service and the set of configuration parameters, such as the model, dataset, number of epochs, and preprocessing parameters. Please feel free to review and adjust the training parameters as needed.

To modify the `user_config.yaml` file, click on the following link to open the file in the Colab workspace (`/content/`): `/content/stm32ai-modelzoo/image_classification/src/user_config.yaml`.

For a custom dataset, in the dataset section, modify:
*   the name and class_names accordingly.
*   training path: `training_path: ../datasets/dataset`

Then, you can tune the other parameters and save the file.

In [ ]:
%cd src
%run stm32ai_main.py

<div id="results">
    <h2>5. Results</h2>
</div>
The trained and quantized models, along with any artifacts, plots, and figures related to the experiments, can be found in the '/content/stm32ai-modelzoo/image_classification/src/experiments_outputs' directory within the Colab workspace (which is located at '/content/').

In [ ]:
import shutil
shutil.make_archive('experiments_outputs', 'zip', 'experiments_outputs')

In [ ]:
# If running on Colab, run this cell to automatically download the outputs.zip file, else download manually.

from google.colab import files
files.download('experiments_outputs.zip')